In [ ]:
import os
import networkx as nx
import numpy as np
from epynet import Network

import sys
sys.path.insert(0, os.path.join('..', 'utils'))
from graph_utils import get_nx_graph
from DataReader import DataReader
from unsupervised_methods import linear_regression

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
wds_id = 'richmond'
obsrat = .8

In [ ]:
path_to_data = os.path.join('..', 'data', 'db_'+wds_id+'_doe_pumpfed_1')
path_to_wds = os.path.join('..', 'water_networks', wds_id+'.inp')

# Loading data
### Loading graph

In [ ]:
wds = Network(path_to_wds)
G_unweighted = get_nx_graph(wds, mode='binary')
L_unweighted = np.array(nx.linalg.laplacianmatrix.laplacian_matrix(G_unweighted).todense())
L_unweighted_normalized = np.array(nx.linalg.laplacianmatrix.normalized_laplacian_matrix(G_unweighted).todense())
G_weighted = get_nx_graph(wds, mode='weighted')
L_weighted = np.array(nx.linalg.laplacianmatrix.laplacian_matrix(G_weighted).todense())
L_weighted_normalized = np.array(nx.linalg.laplacianmatrix.normalized_laplacian_matrix(G_weighted).todense())

### Loading signal

In [ ]:
reader = DataReader(path_to_data, n_junc=len(wds.junctions.uid), obsrat=obsrat, seed=None)
X_complete, _, _ = reader.read_data(
    dataset = 'tst',
    varname = 'junc_heads',
    rescale = 'standardize',
    cover = False
)
X_sparse, bias, scale = reader.read_data(
    dataset = 'tst',
    varname = 'junc_heads',
    rescale = 'standardize',
    cover = True
)

# Graph signal processing
### Smoothness

In [ ]:
X = X_complete[:,:,0].T
smoothness_unweighted = np.dot(X.T, np.dot(L_unweighted, X)).trace()
smoothness_weighted = np.dot(X.T, np.dot(L_weighted, X)).trace()

In [ ]:
print('Smoothness with unweighted Laplacian: {:.0f}.'.format(smoothness_unweighted))
print('Smoothness with weighted Laplacian: {:.0f}.'.format(smoothness_weighted))

### Spectrum

In [ ]:
eigvals_weighted = np.linalg.eigvals(L_weighted_normalized).real
eigvals_unweighted = np.linalg.eigvals(L_unweighted_normalized).real

In [ ]:
plt.bar(np.arange(len(eigvals_weighted)), eigvals_weighted)

In [ ]:
plt.bar(np.arange(len(eigvals_weighted)), eigvals_unweighted)

# Signal reconstruction
### Linear regression
##### Own solution
Based on the paper of Belkin et al.: [https://doi.org/10.1007/978-3-540-27819-1_43](https://doi.org/10.1007/978-3-540-27819-1_43).

In [ ]:
X_hat = linear_regression(L_weighted, X_sparse)

#####  Comparison with Yiye Jiang's solution
Published in [https://arxiv.org/abs/2004.11815](https://arxiv.org/abs/2004.11815).

In [ ]:
sys.path.insert(0, os.path.join('..', '..', 'sensor_selection_NTS'))
from utils import ReLin0

In [ ]:
I = np.where(X_sparse[0,:,1] == 0)[0]
Ic = np.array(list(set(np.arange(X_complete.shape[1]))-set(I)), dtype=int)
Ic.sort()
Sigma_hat = L_weighted
X = X_complete[:,Ic][:,:,0]
Y = X_complete[:,I][:,:,0]

In [ ]:
Y_hat, _ = ReLin0(I, Sigma_hat, X, Y)

##### Comparison

In [ ]:
Y = Y*scale+bias
Y_hat = Y_hat*scale+bias
X_hat = X_hat*scale+bias

In [ ]:
Y[:5,:]

In [ ]:
Y_hat[:5,:]

In [ ]:
X_hat[:5,I]

In [ ]:
print(np.linalg.norm(Y-Y_hat)/np.shape(X_sparse)[0])
print(np.mean(Y-Y_hat))
print(np.std(Y-Y_hat))

In [ ]:
print(np.linalg.norm(Y-X_hat[:,I])/np.shape(X_sparse)[0])
print(np.mean(Y-X_hat[:,I]))
print(np.std(Y-X_hat[:,I]))